In [ ]:
#匯入套件
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import random,requests,re
import matplotlib.pyplot as plt

In [ ]:
#台股上市公司代碼清單
res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
df = pd.read_html(res.text)[0][2][1:]+".TW"

In [ ]:
#爬取股價資訊及生成最大的共變異數矩陣
port = random.choices(df,k=100)
mydata = pd.DataFrame()
#爬取股價資訊(需要一點時間)
for t in range(0,100):
    mydata[port[t]] = wb.DataReader(port[t],data_source ='yahoo', start ='2020-10-1', end ='2021-10-1')['Adj Close']

In [ ]:
#計算每日報酬
port_returns = np.log(mydata / mydata.shift(1))
#共變異數矩陣
cov_matrix = port_returns.cov() 
cov_matrix

In [ ]:
#計算投組標準差
vol_list = []
for i in range(1,101):
    var_bar = (np.diag(cov_matrix.iloc[0:i,0:i]).mean())/i
    cov_bar = sum(sum(np.triu(cov_matrix.iloc[0:i,0:i],1)))
    port_vol = (var_bar+cov_bar/(i**2))**0.5
    vol_list.append(port_vol)
    
vol_list

In [ ]:
#畫圖
plt.xlabel("Number of assets")
plt.ylabel("Risk of portfolio")
plt.title("The Effect of Diversification")
plt.xticks(np.arange(10,101,10))
plt.plot(vol_list)